<a href="https://colab.research.google.com/github/Jungyoonlim/middlemarch/blob/main/Middlemarch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Middlemarch Project.**

Objective: Break down George Eliot's Middlemarch into relevant story metadata.

In [ ]:
from google.colab import files

uploaded = files.upload()

In [2]:
file_path = 'middlemarch.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    middlemarch_text = file.read()

print("Text loaded successfully.")


Text loaded successfully.


In [3]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

# Assuming 'middlemarch_text' contains the entire text of Middlemarch

# Step 1: Remove Gutenberg Headers and Footers
# The headers/footers vary, but often include phrases like "Project Gutenberg"
start_marker = "PRELUDE."
end_marker = "THE END"

start = middlemarch_text.find(start_marker)
end = middlemarch_text.find(end_marker)

# Extract only the main text
main_text = middlemarch_text[start:end]

# Step 2: Tokenization
# Split into sentences
sentences = sent_tokenize(main_text)

# Split into words
words = word_tokenize(main_text)

print(f"Number of sentences: {len(sentences)}")
print(f"Number of words: {len(words)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of sentences: 11900
Number of words: 370620


Named Entity Recognition

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.max_length = len(main_text)
doc = nlp(main_text)

character_entities = set()

for entity in doc.ents:
  if entity.label_ == 'PERSON':
    if not any(char.isdigit() for char in entity.text):
      character_entities.add(entity.text.strip())

for character in character_entities:
  print(character)

Edward Casaubon
Humphrey
Wrench
Gog
Lowick Manor
Cranch
delight,—which
Der
Oliver
Mengan
Baldwin
Christy
Que de voir d’héritiers
SHAKESPEARE
Edinburgh
findeth
Will
Ladislaw
Spanning
Adolf
Fred felt
Henrietta Noble
John Milton
Briggs
Mizraim
Mrs Cadwallader
Lyons
Mythologies
Lydgate
Ware
Simmons
Waverley
Chichely
Philémon
Finer
Walter Scott
Robert
Taft
Adam
aimés dès
Lucy
Glasgow
Laura
Mrs Dollop
emphatically,—“she
Thrice
Solomon
Henry V
Keck
Peel
Rectory
Vicar
Fred and Mary
Balstrode
”—GUIZOT
Nancy Nash
Miss Farebrother
Ellen
Implacable
Lowick
Limp
Mary re-
Laennec
Bulstrode
Library
Jove
Hiram Ford
Mary saw Fred
messenger
awaken
Rigg Featherstone
Fred forsaken
Pseudodoxia Epidemica
Sprague
Pippin
Vuol la sua stagione
half nymph
Mordaunt Merton
Dill
Dorothea’s
Ba-Lamb
Leeds
Rembrandt
Bill
Despond
Fitchett
Sappho
Mozart
Joanna
nosotros viene sobre
John
Jonas
Peacock
rick
Bob
Sophia Primrose
Winifred
Callum
Resolve
Peter
Featherstone
Dodo
Fred Vincy’s
Green Dragon
Mary
Reason
Will Ladisla

In [5]:
from textblob import TextBlob

sentence = TextBlob(sentences[0])
print(sentence.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)


Summary Generation

In [6]:
!pip install sumy

Topic Modeling

Text Summarization

In [10]:
import nltk

In [8]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer

num_sentences_summary = 10

parser = PlaintextParser.from_string(main_text, Tokenizer("english"))
summarizer_list=("TextRankSummarizer:","LexRankSummarizer:","LuhnSummarizer:","LsaSummarizer") #list of summarizers
summarizers = [TextRankSummarizer(), LexRankSummarizer(), LuhnSummarizer(), LsaSummarizer()]

for i, summarizer in enumerate(summarizers):
  print(summarizer_list[i])
  for sentence in summarizer(parser.document, num_sentences_summary):
    print(sentence)
  print("-" * 30)


TextRankSummarizer:


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-a0793d7cbaab>", line 16, in <cell line: 14>
    for sentence in summarizer(parser.document, num_sentences_summary):
  File "/usr/local/lib/python3.10/dist-packages/sumy/summarizers/text_rank.py", line 40, in __call__
    ratings = self.rate_sentences(document)
  File "/usr/local/lib/python3.10/dist-packages/sumy/summarizers/text_rank.py", line 49, in rate_sentences
    matrix = self._create_matrix(document)
  File "/usr/local/lib/python3.10/dist-packages/sumy/summarizers/text_rank.py", line 69, in _create_matrix
    rating = self._rate_sentences_edge(words_i, sentences_as_words[j])
  File "/usr/local/lib/python3.10/dist-packages/sumy/summarizers/text_rank.py", line 89, in _rate_sentences_edge
    rank = sum(words2.count(w) for w in words1)
  File "/usr/local/lib/python3

TypeError: ignored

Topic Modeling

- Preprocess and Tokenize the Text
- Create Dictionary and Corpus
- LDA for Topic Modeling

In [11]:
processed_text = preprocess(main_text)

print(processed_text[:10])

NameError: ignored

In [12]:
import re
# Adjust the regular expression to match the chapter pattern exactly
chapter_pattern = r'\nCHAPTER [IVXLCDM]+\.'  # This includes the newline character and period

# Use re.split to split the text into chapters
chapters = re.split(chapter_pattern, main_text)

chapters = chapters[1:]

# Filter out any empty strings that might have been created during the split
chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

print(f"Total chapters found: {len(chapters)}")
for i in range(3):
    print(f"Start of chapter {i+1}: {chapters[i][:100]}")  # Preview first 100 characters

Total chapters found: 86
Start of chapter 1: Since I can do no good because a woman,
Reach constantly at something that is near it.
             
Start of chapter 2: “‘Dime; no ves aquel caballero que hacia nosotros viene sobre un
caballo rucio rodado que trae puest
Start of chapter 3: “Say, goddess, what ensued, when Raphael,
The affable archangel . . .
                    Eve
The st


In [14]:
from gensim.corpora import Dictionary

dictionary = Dictionary([main_])

dictionary.filter_extremes(no_below=5, no_above=0.7, keep_n=50000)

corpus = [dictionary.doc2bow(text) for text in [processed_text]]

NameError: ignored


Fine-Tune Models
- Narrative Generation and Understanding
- Custom NER Model


In [15]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
fill_mask_pipe = pipeline("fill-mask", model="bert-base-uncased")

# Example usage
masked_text = "The main character in Middlemarch is [MASK]."
predictions = fill_mask_pipe(masked_text)

for prediction in predictions:
    print(prediction)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'score': 0.037618719041347504, 'token': 4300, 'token_str': 'arthur', 'sequence': 'the main character in middlemarch is arthur.'}
{'score': 0.013670222833752632, 'token': 2848, 'token_str': 'peter', 'sequence': 'the main character in middlemarch is peter.'}
{'score': 0.013061015866696835, 'token': 2852, 'token_str': 'dr', 'sequence': 'the main character in middlemarch is dr.'}
{'score': 0.011167364194989204, 'token': 17001, 'token_str': 'sgt', 'sequence': 'the main character in middlemarch is sgt.'}
{'score': 0.009783466346561909, 'token': 2198, 'token_str': 'john', 'sequence': 'the main character in middlemarch is john.'}


In [3]:
middlemarch_paragraphs = middlemarch_text.split('\n\n')  # Assuming two newlines separate paragraphs

NameError: ignored

In [2]:
from transformers import BartTokenizer
from datasets import Dataset

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

def process_data_to_model_inputs(batch):
    # Tokenize the inputs and labels
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    return inputs

# Create a dataset object
dataset = Dataset.from_dict({'text': middlemarch_paragraphs})
dataset = dataset.map(process_data_to_model_inputs, batched=True)

NameError: ignored

In [ ]:
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
!pip install transformers[torch] -U

In [ ]:
!pip install torch -U

In [1]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

trainer.train()


NameError: ignored

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="./fine_tuned_bart", tokenizer=tokenizer)

generated_text = generator("Example prompt", max_length=50)
print(generated_text[0]['generated_text'])

Further Metadata Extraction
- Character Relationships
- Plot Analysis
- Style and Tone Analysis